In [5]:
%matplotlib inline
import sys
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../../')
sys.path.append('../../../../')
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

from PyFin.api import *
from alphamind.api import *
from conf.models import *
from conf.config import*
from data.engines.model import Record
from alphamind.execution.naiveexecutor import NaiveExecutor
from stacking import factor_store, feature_list
from optimization.bayes_optimization_xgb import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)

from data.engines.sqlengine import SQLEngine

weekly = 's2'  # s1, s2,  s3, s4, s5, s_1
record_engine = SQLEngine('sqlite:///./{}/real_tune_record_without_alpha.db'.format(weekly))


In [6]:
# ref_date_pre = '2019-12-27'

previous_pos = record_engine.fetch_record_meta(Record, trade_date=None)
previous_pos

,trade_date,adjust_date,weight,industry,er,code
0,2019-12-24,2019-12-23,0.027590,计算机,0.013777,66
1,2019-12-24,2019-12-23,0.026359,电子,0.031855,603501
2,2019-12-24,2019-12-23,0.025949,计算机,0.017539,600536
3,2019-12-24,2019-12-23,0.025918,机械设备,0.016362,300450
4,2019-12-24,2019-12-23,0.024555,传媒,0.030923,300315
5,2019-12-24,2019-12-23,0.024555,农林牧渔,0.007312,600298
6,2019-12-24,2019-12-23,0.024391,计算机,0.014319,300168
7,2019-12-24,2019-12-23,0.024381,电子,0.018951,2371
8,2019-12-24,2019-12-23,0.024155,食品饮料,0.014721,2507
9,2019-12-24,2019-12-23,0.024114,医药生物,0.006363,600673


In [ ]:
previous_pos['adjust_date'] = previous_pos.trade_date
previous_pos['trade_date'] = previous_pos.trade_date + timedelta(days=1)
previous_pos

In [ ]:
record_engine.write_data('pos_record', previous_pos, if_exists='replace')

In [ ]:
# record_engine.fetch_record('pos_record')
previous_pos = record_engine.fetch_record_meta(Record, trade_date=None)


In [ ]:
previous_pos